In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6693, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6681, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6699, 9)

fin empresas y comienzo de rescate de pacientes


In [5]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1581104, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [6]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1096574, 4)

In [7]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1096574, 12)

Carga de Archivo carga

In [166]:
nuevos = ("F:/Empresas SF/reportes/carga.xlsx")
campaña ='Pesco'
Pais = 'Chile'




Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

Pacientes_Archivo.head()                                       

,Empleado - RUT,Estado,rut1,Unnamed: 3,Unnamed: 4,Empleado - Nombre Completo,apellidos,nombres,Empresa - Nombre Empresa,cargo,...,Empleado - Estado,Trabajo - Fecha Ingreso Compañía,Trabajo - Centro de Costo,Trabajo - Nombre Supervisor,nacimiento,sexo,Empleado - Nacionalidad,celular,email,rut
0,12857704-1,12857704-1,12857704-1,1.0,NaN,Narvaez Molina Rodrigo,Narvaez Molina,Rodrigo,Peña Spoerer y Cía SA,JEFE DE SUCURSAL,...,Activo,2020-03-02,428,Jaime Eduardo Andrews Soto,1975-04-17,M,Chilena,9.991902e+08,narvaezrodrigo@hotmailcom,128577041
1,17253870-3,17253870-3,17253870-3,1.0,NaN,Steck Silva Stephanie Francesca,Steck Silva,Stephanie Francesca,Peña Spoerer y Cía SA,EJECUTIVO COMERCIAL,...,Activo,2021-05-03,40411,Antonio Del Rio Hoffmann,1989-10-15,F,Chilena,5.699477e+10,stephaniesteck@hotmailcom,172538703
2,14124888-k,14124888-k,14124888-k,1.0,NaN,Zuñiga Quiroz Gloria Macarena,Zuñiga Quiroz,Gloria Macarena,Peña Spoerer y Cía SA,SUPERVISOR_,...,Activo,2021-05-17,203,Mario Antonio Contreras Fuentes,1981-01-22,F,Chilena,5.699837e+10,GLORIA_MZQ@HOTMAILCOM,14124888k
3,18508342-k,18508342-k,18508342-k,1.0,NaN,Espejo Castillo Jose Armando,Espejo Castillo,Jose Armando,Peña Spoerer y Cía SA,TECNICO DE SERVICIO,...,Activo,2021-01-11,301,Rodrigo Narvaez Molina,1993-12-22,M,Chilena,9.663543e+08,joseespejo221293@gmailcom,18508342k
4,16574011-4,16574011-4,16574011-4,1.0,NaN,Vidal Osorio Leonora Paz,Vidal Osorio,Leonora Paz,Peña Spoerer y Cía SA,ANALISTA CONTABLE SENIOR,...,Activo,2021-06-14,203,Gloria Macarena Zuñiga Quiroz,1987-02-17,F,Chilena,5.694451e+10,vidalleonora@gmailcom,165740114


In [167]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_908\1036101616.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empleado - RUT,Estado,rut1,Unnamed: 3,Unnamed: 4,Empleado - Nombre Completo,apellidos,nombres,Empresa - Nombre Empresa,cargo,...,Empleado - Estado,Trabajo - Fecha Ingreso Compañía,Trabajo - Centro de Costo,Trabajo - Nombre Supervisor,nacimiento,sexo,Empleado - Nacionalidad,celular,email,rut
0,12857704-1,12857704-1,12857704-1,1.0,NaN,Narvaez Molina Rodrigo,Narvaez Molina,Rodrigo,Peña Spoerer y Cía SA,JEFE DE SUCURSAL,...,Activo,2020-03-02,428,Jaime Eduardo Andrews Soto,1975-04-17,M,Chilena,9.991902e+08,narvaezrodrigo@hotmailcom,128577041
1,17253870-3,17253870-3,17253870-3,1.0,NaN,Steck Silva Stephanie Francesca,Steck Silva,Stephanie Francesca,Peña Spoerer y Cía SA,EJECUTIVO COMERCIAL,...,Activo,2021-05-03,40411,Antonio Del Rio Hoffmann,1989-10-15,F,Chilena,5.699477e+10,stephaniesteck@hotmailcom,172538703
2,14124888-k,14124888-k,14124888-k,1.0,NaN,Zuñiga Quiroz Gloria Macarena,Zuñiga Quiroz,Gloria Macarena,Peña Spoerer y Cía SA,SUPERVISOR_,...,Activo,2021-05-17,203,Mario Antonio Contreras Fuentes,1981-01-22,F,Chilena,5.699837e+10,GLORIA_MZQ@HOTMAILCOM,14124888K
3,18508342-k,18508342-k,18508342-k,1.0,NaN,Espejo Castillo Jose Armando,Espejo Castillo,Jose Armando,Peña Spoerer y Cía SA,TECNICO DE SERVICIO,...,Activo,2021-01-11,301,Rodrigo Narvaez Molina,1993-12-22,M,Chilena,9.663543e+08,joseespejo221293@gmailcom,18508342K
4,16574011-4,16574011-4,16574011-4,1.0,NaN,Vidal Osorio Leonora Paz,Vidal Osorio,Leonora Paz,Peña Spoerer y Cía SA,ANALISTA CONTABLE SENIOR,...,Activo,2021-06-14,203,Gloria Macarena Zuñiga Quiroz,1987-02-17,F,Chilena,5.694451e+10,vidalleonora@gmailcom,165740114


In [168]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_908\2096211457.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_908\2096211457.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [169]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(249, 32)

In [170]:
Pacientes_con_id.head()

,Empleado - RUT,Estado,rut1,Unnamed: 3,Unnamed: 4,Empleado - Nombre Completo,apellidos,nombres,Empresa - Nombre Empresa,cargo,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,12857704-1,12857704-1,12857704-1,1.0,NaN,Narvaez Molina Rodrigo,Narvaez Molina,Rodrigo,Peña Spoerer y Cía SA,JEFE DE SUCURSAL,...,True,0018c00002egsNOAAY,128577041,Rodrigo Narvaez Molina,RUT,None,None,None,Hombre,None
1,17253870-3,17253870-3,17253870-3,1.0,NaN,Steck Silva Stephanie Francesca,Steck Silva,Stephanie Francesca,Peña Spoerer y Cía SA,EJECUTIVO COMERCIAL,...,True,0018c00002egsNQAAY,172538703,Stephanie Francesca Steck Silva,RUT,None,None,None,Mujer,None
2,14124888-k,14124888-k,14124888-k,1.0,NaN,Zuñiga Quiroz Gloria Macarena,Zuñiga Quiroz,Gloria Macarena,Peña Spoerer y Cía SA,SUPERVISOR_,...,True,0018c00002amLUtAAM,14124888K,Gloria Macarena Zuñiga Quiroz,RUT,None,None,None,Mujer,998369653
3,18508342-k,18508342-k,18508342-k,1.0,NaN,Espejo Castillo Jose Armando,Espejo Castillo,Jose Armando,Peña Spoerer y Cía SA,TECNICO DE SERVICIO,...,True,0018c00002egsMmAAI,18508342K,Jose Armando Espejo Castillo,RUT,None,None,None,Otros,None
4,16574011-4,16574011-4,16574011-4,1.0,NaN,Vidal Osorio Leonora Paz,Vidal Osorio,Leonora Paz,Peña Spoerer y Cía SA,ANALISTA CONTABLE SENIOR,...,True,0018c00002amT5jAAE,165740114,Leonora Vidal Osorio,RUT,None,None,None,Mujer,None


In [171]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [172]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
       
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "Mexico"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000004cvS7EAI'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,PersonTitle,PersonDepartment,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [173]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper()
clinica_virtual['firstName'] = Pacientes_con_id['nombres']
clinica_virtual['lastName'] = Pacientes_con_id['apellidos']
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'Talo':'36',
'TRANSVIP':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40',
'Southbrige':'41',
'Construmart':'42',
'BICE Vida':'43'

}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [174]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
160130,a1Y8c00000C6VeNEAV,0018c00002amRRLAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
160131,a1Y8c00000C6VeOEAV,0018c00002amRzAAAU,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
160132,a1Y8c00000C6VePEAV,0018c00002amLUtAAM,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
160133,a1Y8c00000C6VeQEAV,0018c00002amSFBAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723
160134,a1Y8c00000C6VeREAV,0018c00002amSFFAA2,a1W8c0000083iWjEAI,Pesco,0018c00002cXJTtAAO,Pesco Holding Pesco Peña Spoerer Y Cia S.A.,Holding Pesco,Pesco,None,None,EMP7399,A-0723


In [175]:
campaña_seleccionada.shape

(323, 12)

In [176]:

if campaña != 'MetLife':
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente"
    )
    pacientes_en_campaña['Paciente__c'] = 'Id Pacientes'
    pacientes_en_campaña['Programas_por_Empresas__c'] = 'Id Programas Por empresa'
    pacientes_en_campaña.to_excel("F:/Empresas SF/reportes/campaña_empresa.xlsx",index=False)

In [177]:
campaña_en_pacientes = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account"
)
campaña_en_pacientes.to_excel("F:/Empresas SF/reportes/archivo_campaña.xlsx",index=False)